# Serpent Input Generator Demo
1. Showcase of functionality from the in progress development of the serpentInputGenerator

In [43]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.lats import lats
from serpentGenerator.data.materialLibrary import MATLIB

## Using pregenerated materials for effeciency
1. users can take advantadge of the pregenerated material library and tweak paramters to their liking
2. users can also use data from pregenerated materials and create their own
3. users can access all the pregenerated materials using the ``list()`` function on the material library ``MATLIB``

In [44]:
list(MATLIB)

['UO2', 'Zr', 'boron', 'lightWater', 'plenum']

## Creating our custom UO2 fuel
1. using the ``vars()`` function we can see the default pregenerated UO2 material parameters 
2. for this example we will be generating a higher enrichement uranium oxide


In [45]:
vars(MATLIB['UO2'])

{'name': 'UO2',
 'isBurn': True,
 'isModer': True,
 'dens': -10.2141,
 'nuclides': array([92235, 92238,  8016], dtype=int64),
 'fractions': array([0.00133134, 0.02218898, 0.04600811]),
 'temp': 0.0,
 'xsLib': '12c',
 'modLib': 'None',
 'color': 'red'}

In [46]:
fuel = MATLIB['UO2']
fuel.set('fractions', np.array([0.001331338626, 0.02218898, 0.04600811])) # 6% Enrichment
fuel.set('xsLib', "12c") #Higher enrichment cores will run at higher temperatures so we shall change the xs library suffix
fuel.set('color','red') #change the color of the fuel for plotting purposes
vars(fuel) #Display updated properties of our new fuel

{'name': 'UO2',
 'isBurn': True,
 'isModer': True,
 'dens': -10.2141,
 'nuclides': array([92235, 92238,  8016], dtype=int64),
 'fractions': array([0.00133134, 0.02218898, 0.04600811]),
 'temp': 0.0,
 'xsLib': '12c',
 'modLib': 'None',
 'color': 'red'}

## Creating a pin using our new fuel
1. a pin element can be created using user created and pre generated materials
2. the user will have to specify the materials and outer radius of each material in an order dependent fashion
2. For this scenario the pin will have 3 regions conisiting of fuel, cladding, and moderator
3. the user will use the newly created fuel and is content using the pregenerated Zr cladding and lightWater moderator


In [47]:
p1 = pin("101", 3) #id = 101, numberOfRegions = 3
p1.set('materials', np.array([fuel, MATLIB['Zr'], MATLIB['lightWater']]))
p1.set('radii', np.array([.45, .47]))
print(p1.toString()) #display the pin element in serpent format 

pin 101
UO2	0.45
Zr	0.47
lightWater




## Creating a standard fuel pin and guide pin
1. A standard fuel pin will be created for comparison with the higher enrichment fuel pin
2. A guide tube pin will be created.
3. the pinObject.``duplicatePin(newPinId)`` method will be used to fascillitate the process
4. all attributes will be the same except the pin id and a change in the inner pin material

In [48]:
p2 = p1.duplicatePin("201")
p3 = p1.duplicatePin("301")
p2.set('materials', np.array([MATLIB['UO2'], MATLIB['Zr'], MATLIB['lightWater']])) #std UO2 Fuel
p3.set('materials', np.array([MATLIB['lightWater'], MATLIB['Zr'], MATLIB['lightWater']])) 


## Creating a pins object for storage
1. pins created by the user can be stored in a pins object which is meant to be a library for pins.

In [49]:
pinLib = pins()
pins1 = [p1, p2, p3]
pinLib.addPins(pins1)
vars(pinLib)

{'npins': 3,
 'pins': {'101': <serpentGenerator.functions.pin.pin at 0x24570fb3a48>,
  '201': <serpentGenerator.functions.pin.pin at 0x24570fc4388>,
  '301': <serpentGenerator.functions.pin.pin at 0x24570fc4908>}}

## Creating a sqaure Lattice
1. For this scenario, the sqaure lattice will be a 17 x 17 lattice, positiioned at the origin with a pitch of 1.260 cm


In [50]:
lat1 = sqLat("l1", 0, 0, 17, 1.260)
vars(lat1)

{'id': 'l1',
 'xo': 0,
 'yo': 0,
 'nelements': 17,
 'pitch': 1.26,
 'map': array([], shape=(1, 0), dtype=float64)}

## Creating the lattice map layout
1. The map can be set using the Object.``setMap(map)`` function
2. The map for a square lattice must be a 2d numpy array sqaure in shape
3. The map array can consist of pin objects for example

In [51]:
latMap1 = np.array([
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p1, p1, p1],
    [p1, p1, p1, p3, p1, p1, p1, p1, p1, p1, p1, p1, p1, p3, p1, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p3, p1, p1, p1, p1, p1, p1, p1, p1, p1, p3, p1, p1, p1],
    [p1, p1, p1, p1, p1, p3, p1, p1, p3, p1, p1, p3, p1, p1, p1, p1, p1],  
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    [p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1, p1],
    ])

lat1.setMap(latMap1)
print(lat1.toString())

l1 0 0  17 1.26
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 101 101 101 301 101 101 301 101 101 301 101 101 101 101 101 
101 101 101 301 101 101 101 101 101 101 101 101 101 301 101 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 301 101 101 301 101 101 301 101 101 301 101 101 301 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 301 101 101 301 101 101 301 101 101 301 101 101 301 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 301 101 101 301 101 101 301 101 101 301 101 101 301 101 101 
101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 
101 101 101 301 101 101 101 101 101 101 101 101 101 301 101 101 101 
101 101 101 101 10

## Creating a duplicate lattice and altering the contents
1. The first lattice created above, used the higher enriched fuel pins.
2. For comparison, a duplicate lattice will be created keeping the same structure except now using the standard fuel pins.
3. A copy/duplicate sqLat object can be made from an exisiting lattice using the Object.``duplicateLat('newLatId')`` function
4. the Object.``replacePin(oldPin, newPin)`` function can be used to replace pins in the lattice map layout

In [52]:
lat2 = lat1.duplicateLat("l2")
lat2.replacePin(p1, p2)
print(lat2.toString())

l2 0 0  17 1.26
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 201 201 201 301 201 201 301 201 201 301 201 201 201 201 201 
201 201 201 301 201 201 201 201 201 201 201 201 201 301 201 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 301 201 201 301 201 201 301 201 201 301 201 201 301 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 301 201 201 301 201 201 301 201 201 301 201 201 301 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 301 201 201 301 201 201 301 201 201 301 201 201 301 201 201 
201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 201 
201 201 201 301 201 201 201 201 201 201 201 201 201 301 201 201 201 
201 201 201 201 20

## Creating a lats object for storage
1. lattices created by the user can be stored in a lats object which is meant to be a library for lattices.

In [53]:
latLib = lats()
lats1 = [lat1, lat2]
latLib.addLats(lats1)
vars(latLib)

{'nlats': 2,
 'lats': {'l1': <serpentGenerator.functions.sqLattice.sqLat at 0x24570fb3588>,
  'l2': <serpentGenerator.functions.sqLattice.sqLat at 0x24570fa83c8>}}